In [1]:
# importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import time

# Extracting just the href links for the product listing from amazon website, so that we can scrape additional data for a product

In [2]:
driver = webdriver.Chrome()

titles, prices, links = [], [], [] # creating 3 lists and storing the data in those lists

# Loop through the pages
for i in tqdm(range(1, 30)):  
    url = f'https://www.amazon.in/s?k=phones&page={i}&crid=1UO0Z6YYHREBN&qid=1724361232&sprefix=phone%2Caps%2C229&ref=sr_pg_{i}'
    driver.get(url)
    
    containers = driver.find_elements(By.CSS_SELECTOR, "div.puisg-col-inner")
    
    for container in containers:
        try:
            name = container.find_element(By.CSS_SELECTOR, "span.a-size-medium.a-color-base.a-text-normal").text
        except:
            name = None
        
        try:
            price = container.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
        except:
            price = None
        
        try:
            link = container.find_element(By.CSS_SELECTOR, "a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal").get_attribute('href')
        except:
            link = None

        if name and link:
            titles.append(name)
            prices.append(price)
            links.append(link)

driver.quit()



100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [02:00<00:00,  4.15s/it]


In [3]:
# Converting it into a dataframe
df1 = pd.DataFrame()
df1['Product'] = titles
df1['Price'] = prices
df1['Links'] = links


In [34]:
df1['Links'][1]

'https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo0MTUwNDMzOTU5NDgwMjk5OjE3MjQzOTg0NTg6c3BfYXRmOjMwMDA1NzU5NDU0MDgzMjo6MDo6&url=%2FApple-iPhone-15-Plus-128%2Fdp%2FB0CHX6X2WW%2Fref%3Dsr_1_2_sspa%3Fcrid%3D1UO0Z6YYHREBN%26dib%3DeyJ2IjoiMSJ9.brV7rG-zDD0MO_wZuT9UcB5RXXhETdhUGWjD_ndeIHrhOEKEaz-Ur6iQZL-bfKcwrZMV5EEzWlB-JAxj_GBJSxDs9h-lU_47gShwVfxdJQRSrUUFfmntJwJI3XMUaOrK8SLdq2eDpHUISJDdmcVPM5T6Cj1FTdSOjvXTl01hgNtaFAwFqOzdRwBgIo5Kntjb_q9aw3FfUxmuGdH4b31a_myPfDx-Nc-sHGwybZ93NL0.IxNxhJQ4m265toZD2YMxqb3TwxDsi4ixlMPoO5CB34U%26dib_tag%3Dse%26keywords%3Dphones%26qid%3D1724398458%26sprefix%3Dphone%252Caps%252C229%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'

In [5]:
df1

,Product,Price,Links
0,"Redmi Note 13 Pro+ (Fusion White, 8GB RAM, 256...","29,499",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Apple iPhone 15 Plus (128 GB) - Blue,"79,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,"Redmi 13C (Stardust Black, 6GB RAM, 128GB Stor...","8,499",https://www.amazon.in/Stardust-Storage-Powered...
3,POCO C65 (Pastel Blue 4GB RAM 128GB Storage),"6,799",https://www.amazon.in/POCO-C65-Pastel-Blue-128...
4,"realme NARZO N65 5G (Deep Green 6GB RAM, 128GB...","12,498",https://www.amazon.in/realme-Storage-Chipset-C...
...,...,...,...
491,Apple iPhone 15 Pro Max (256 GB) - White Titanium,"1,46,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
492,Apple iPhone 15 (128 GB) - Pink,"71,290",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
493,Apple iPhone 13 (256GB) - Midnight,"61,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
494,INFLUX Mobile Phone SE 16GB Grey Compatible wi...,"10,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [6]:
df1.to_csv('df1.csv')

# Extracting main informations form each Href links 

In [6]:
#df1 = pd.read_csv('df1.csv')

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time

name = []
brand = []
os = []
ram = []
memory = []
soc = []


for link in tqdm(df1['Links']):   # Looping through the links
    url = link
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    
    try:
        a = driver.find_element(By.CSS_SELECTOR, 'span.a-size-large.product-title-word-break')
        name.append(a.text)
    except:
        name.append(None)
        

    curr_brand = None      # Initialize default values for the current product so that if a value is not found then it will NA
    curr_os = None
    curr_ram = None
    curr_memory = None
    curr_soc = None

    try:
        b = driver.find_elements(By.CSS_SELECTOR, 'td.a-span3')  # used try and except to handle errors if any occurs
        c = driver.find_elements(By.CSS_SELECTOR, 'td.a-span9')

        for i in range(len(b)):
            text = b[i].text.strip().lower()
            value = c[i].text.strip()

            if text == 'brand':
                curr_brand = value
            elif text == 'cpu model':
                curr_soc = value
            elif text == 'operating system':
                curr_os = value
            elif text == 'ram memory installed size':
                curr_ram = value
            elif text == 'memory storage capacity':
                curr_memory = value

    except:
        pass

    # Append the collected data into respective list
    brand.append(curr_brand)
    os.append(curr_os)
    ram.append(curr_ram)
    memory.append(curr_memory)
    soc.append(curr_soc)
    
    driver.quit()


100%|██████████████████████████████████████████████████████████████████████████████| 496/496 [2:49:37<00:00, 20.52s/it]

Names: ["Redmi Note 13 Pro+ (Fusion White, 8GB RAM, 256GB Storage) | World's first Mediatek 7200 Ultra 5G | 200MP Hi-Res Camera | 1.5K Curved AMOLED | 120W HyperCharge", 'Apple iPhone 15 Plus (128 GB) - Blue', 'Redmi 13C (Stardust Black, 6GB RAM, 128GB Storage) | Powered by 4G MediaTek Helio G85 | 90Hz Display | 50MP AI Triple Camera', 'POCO C65 (Pastel Blue 4GB RAM 128GB Storage)', "realme NARZO N65 5G (Deep Green 6GB RAM, 128GB Storage) India's 1st D6300 5G Chipset | Ultra Slim Design | 120Hz Eye Comfort Display | 50MP AI Camera| Charger in The Box", 'Redmi 13 5G, Hawaiian Blue, 6GB+128GB | India Debut SD 4 Gen 2 AE | 108MP Pro Grade Camera | 6.79in Largest Display in Segment', "realme NARZO N65 5G (Amber Gold 6GB RAM, 128GB Storage) India's 1st D6300 5G Chipset | Ultra Slim Design | 120Hz Eye Comfort Display | 50MP AI Camera| Charger in The Box", 'realme NARZO 70 5G（Ice Blue,6GB RAM, 128GB Storage | Dimensity 7050 5G Chipset | 120Hz AMOLED Display | 50MP Primary Camera | 45W Charger

In [61]:
# Again converting it into a dataframe
df2 = pd.DataFrame()
df2['Product'] = name
df2['Brand']=brand
df2['Os']=os
df2['Ram']=ram
df2['Memory']=memory


In [62]:
df2

,Product,Brand,Os,Ram,Memory
0,"Redmi Note 13 Pro+ (Fusion White, 8GB RAM, 256...",Redmi,Android 13.0,8 GB,256 GB
1,Apple iPhone 15 Plus (128 GB) - Blue,Apple,iOS,None,128 GB
2,"Redmi 13C (Stardust Black, 6GB RAM, 128GB Stor...",Xiaomi,"MIUI 14, Android 13.0",6 GB,128 GB
3,POCO C65 (Pastel Blue 4GB RAM 128GB Storage),POCO,Android 13.0,4 GB,None
4,"realme NARZO N65 5G (Deep Green 6GB RAM, 128GB...",realme,Android 14,6 GB,128 GB
...,...,...,...,...,...
491,Apple iPhone 15 Pro Max (256 GB) - White Titanium,Apple,iOS,None,256 GB
492,Apple iPhone 15 (128 GB) - Pink,Apple,iOS,None,128 GB
493,Apple iPhone 13 (256GB) - Midnight,Apple,iOS 14,None,256 GB
494,INFLUX Mobile Phone SE 16GB Grey Compatible wi...,INFLUX,iOS 9,None,16 GB


In [68]:
df2.to_csv('df2.csv')

In [65]:
final=df1.merge(df2 , on='Product', how='right') # finally merging the two datasets on the product name and getting final df

In [66]:
final

,Product,Price,Links,Brand,Os,Ram,Memory
0,"Redmi Note 13 Pro+ (Fusion White, 8GB RAM, 256...","29,499",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Redmi,Android 13.0,8 GB,256 GB
1,"Redmi Note 13 Pro+ (Fusion White, 8GB RAM, 256...","29,499",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Redmi,Android 13.0,8 GB,256 GB
2,Apple iPhone 15 Plus (128 GB) - Blue,"79,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Apple,iOS,None,128 GB
3,"Redmi 13C (Stardust Black, 6GB RAM, 128GB Stor...","8,499",https://www.amazon.in/Stardust-Storage-Powered...,Xiaomi,"MIUI 14, Android 13.0",6 GB,128 GB
4,POCO C65 (Pastel Blue 4GB RAM 128GB Storage),"6,799",https://www.amazon.in/POCO-C65-Pastel-Blue-128...,POCO,Android 13.0,4 GB,None
...,...,...,...,...,...,...,...
959,Amkette iGrip Secure Plus Bike Phone Holder fo...,"1,099",https://aax-eu.amazon.in/x/c/JH4XBdAP7HzOYDBlR...,Amkette,None,None,None
960,Amkette iGrip Secure Plus Bike Phone Holder fo...,"1,099",https://aax-eu.amazon.in/x/c/JAYE5MrOd4ZAjIvtQ...,Amkette,None,None,None
961,Amkette iGrip Secure Plus Bike Phone Holder fo...,"1,099",https://aax-eu.amazon.in/x/c/JHnEIOntY4hSoXDRG...,Amkette,None,None,None
962,Amkette iGrip Secure Plus Bike Phone Holder fo...,"1,099",https://aax-eu.amazon.in/x/c/JOXDe2jvzV2QQMltE...,Amkette,None,None,None


In [69]:
final.to_csv("final.csv")